# 閉包模型鏈 (Closure Model Chains) - Gemini 版本

本範例展示如何使用 Google Gemini 在 RunnableLambda 中通過閉包方式直接調用模型。

## 🎯 學習目標
- 理解閉包在 LangChain 中的應用
- 掌握在自定義函數中調用模型的技巧
- 學習完全控制處理流程的方法

## 📝 核心概念
閉包模型鏈通過在 RunnableLambda 函數內部捕獲外部的模型實例，實現靈活的自定義處理邏輯。

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")
print("✅ Gemini 模型已建立")

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

def format_with_quality_check(reply):
    formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
"""
    
    quality_check_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是郵件品質審查專家。請評估這封郵件並給出1-10分評分和簡短評語。"),
        ("human", "評估這封郵件：\n\n{email}")
    ])
    
    quality_result = quality_check_prompt.format_prompt(email=formatted_reply)
    quality_score = model.invoke(quality_result.to_messages())
    
    return f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{formatted_reply}

{'='*60}
📊 AI 品質檢查結果：
{'='*60}
{quality_score.content}
"""

closure_chain = prompt_template | model | StrOutputParser() | RunnableLambda(format_with_quality_check)
print("✅ 閉包模型鏈已建立")

In [ ]:
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

result = closure_chain.invoke({"email_content": customer_email})
print(result)

## 📊 總結

閉包模型鏈提供了極高的靈活度，適合需要複雜自定義邏輯的場景。

### 🔗 相關技術
- [串聯模型鏈](5_chains_sequential_model_gemini.ipynb)
- [並行模型鏈](8_chains_parallel_model_gemini.ipynb)
- [動態提示鏈](9_chains_dynamic_prompt_gemini.ipynb)